In [4]:
import json
import numpy as np
import tensorflow as tf
import keras
from keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# WITH PREPROCESSING

In [5]:
ds_train = image_dataset_from_directory('../data/french_bird_db_50_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=64)

ds_test = image_dataset_from_directory('../data/french_bird_db_50_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=64)

Found 5000 files belonging to 50 classes.
Using 4000 files for training.
Found 5000 files belonging to 50 classes.
Using 1000 files for validation.


In [6]:
class_names = ds_train.class_names
num_classes = len(ds_train.class_names)
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

In [7]:
# preprocessing
preprocess_input = tf.keras.applications.resnet50.preprocess_input

ds_train_preprocessed = ds_train.map(lambda x, y: (preprocess_input(x), y))
ds_test_preprocessed = ds_test.map(lambda x, y: (preprocess_input(x), y))


In [8]:
tf.config.list_physical_devices()

# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

x_3 = Dense(num_classes, activation='relu')(x_2)

# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]


# Entraîner le modèle sur votre dataset avec le callback personnalisé
history = model.fit(ds_train_preprocessed, epochs=10, callbacks=callbacks, validation_data=ds_test_preprocessed)

Epoch 1/10


2024-07-03 15:54:57.074186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


63/63 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - loss: 3.5502 - sparse_categorical_accuracy: 0.1395 - val_loss: 2.1426 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - loss: 1.7419 - sparse_categorical_accuracy: 0.6069 - val_loss: 1.2994 - val_sparse_categorical_accuracy: 0.6590
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - loss: 1.0402 - sparse_categorical_accuracy: 0.7627 - val_loss: 1.0058 - val_sparse_categorical_accuracy: 0.7360
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - loss: 0.7313 - sparse_categorical_accuracy: 0.8294 - val_loss: 0.8393 - val_sparse_categorical_accuracy: 0.7850
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - loss: 0.5620 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.7889 - val_sparse_categorical_accuracy: 0.7800
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - loss: 0.4507 - sparse_categorical_accuracy: 0.9051 - val_loss: 0.7293 - val_sparse_categorical_accuracy: 0.8060
Epoch 7/10
63/63 ━━━━

In [13]:
model.save_weights('current_model.weights.h5')

In [39]:
image_test = keras.utils.load_img(
    '../data/french_bird_db_50_100/apus_apus/apus_apus_10.jpg',
)

image_array = keras.utils.img_to_array(image_test)

image_processed = preprocess_input(image_array)
# ds_train_preprocessed = ds_train.map(lambda x, y: (preprocess_input(x), y))
print(image_processed.shape)

test = np.argmax(model.predict(np.expand_dims(image_processed, axis=0)))

print(test, class_names[test])

(300, 300, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
0 accipiter_gentilis


In [14]:
for image_batch, labels_batch in ds_train_preprocessed:
  batch_pred = model.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  print(class_pred)
  print(labels_batch)
  break

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
[41 32 36 44 35 31  4 28 33 12 13  9 14 31 28 23  7 41 21 10 11  8 46  5
  1 16 12 13 39 46 25  2 36 48 28  5 14 19 34 11 43 25 41 20  8 15 48 27
 35  6 26 22 14 28 37 18  5 14  2 30 39 13 28 31]
tf.Tensor(
[41 32 36 44 35 31  4 28 33 12 13  9 14 31 28 23  7 41 21 10 11  8 46  5
  1 16 12 13 39 46 25  2 36 48 28  5 14 19 34 11 43 25 41 20  8 15 48 27
 35  6 26 22 14 28 37 18  5 14  2 30 39 13 28 31], shape=(64,), dtype=int32)


In [15]:
total = 0
for i, (image_batch, labels_batch) in enumerate(ds_test_preprocessed):
  batch_pred = model.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  res = sum((class_pred == labels_batch).numpy()) / len(class_pred)
  total += res
  
  print('batch No {} with result: '.format(i), res * 100, '%')
  
print('FINAL RES = ', total / len(ds_test_preprocessed))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step
batch No 0 with result:  89.0625 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 675ms/step
batch No 1 with result:  76.5625 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step
batch No 2 with result:  75.0 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 668ms/step
batch No 3 with result:  84.375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step
batch No 4 with result:  87.5 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
batch No 5 with result:  76.5625 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 767ms/step
batch No 6 with result:  84.375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
batch No 7 with result:  75.0 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 691ms/step
batch No 8 with result:  82.8125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step
batch No 9 with result:  85.9375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
batch No 10 with result:  82.8125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms/step
batch No 11 with result:  73.4375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 685ms/step
batch No 12 with result:  73.4375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 672ms/step
batch N

2024-07-03 16:18:06.644930: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
